In [ ]:
#import required libraries
import matplotlib.pyplot as plt
import numpy as np
import shutil
%matplotlib inline
import os
import skimage.io as io
import os

In [ ]:
import skimage
def _read_img(image_path):
    img = skimage.io.imread(image_path, plugin='tifffile')
    return img

In [ ]:
imdir = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\MC_road_GT\GT_MC_road.tif"

In [ ]:
IM = io.imread(imdir)

In [ ]:
IM.shape

In [ ]:
IM.dtype

In [ ]:
np.unique(IM)

In [ ]:
IM[IM==3] =1
IM[IM==4] =3
IM[IM==5] =4
IM[IM==6] =5

In [ ]:
np.unique(IM, return_counts = True)

In [ ]:
np.count_nonzero(IM == 6)

In [ ]:
np.count_nonzero(IM == 5)

In [ ]:
np.count_nonzero(IM == 4)

In [ ]:
np.count_nonzero(IM == 3)

In [ ]:
np.count_nonzero(IM == 2)

In [ ]:
np.count_nonzero(IM == 1)

In [ ]:
np.count_nonzero(IM == 0)

In [ ]:
palette ={ 0 : (0,0,0), # NoData
           1 : (77, 255, 0), #Ausgebaute Allwetterstrasse
           2 : (204, 0, 0),  #Eisenbahn
           3 : (230, 128, 0), #Fußweg
           4 : (255, 0, 0),  #Karawanenweg
           5 : (0, 204, 242), #Saumweg
         }

invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(arr_2d, palette=palette):
    """ Numeric labels to RGB-color encoding """
    arr_3d = np.zeros((arr_2d.shape[0], arr_2d.shape[1], 3), dtype=np.uint8)

    for c, i in palette.items():
        m = arr_2d == c
        arr_3d[m] = i

    return arr_3d

def convert_from_color(arr_3d, palette=invert_palette):
    """ RGB-color encoding to grayscale labels """
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    return arr_2d

In [ ]:
colored_mask = convert_to_color(IM)

In [ ]:
plt.imshow(colored_mask)

In [ ]:
save_path = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\MC_road_GT\GT_colored.tif"

In [ ]:
io.imsave(save_path, colored_mask) #no CRS 

In [ ]:
# helper function for data visualization
def visualize_in_one_row(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    

def visualize(image):
    # Divide all values by 65535 so we can display the image using matplotlib
    image = image / 65535
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image, cmap = 'Greys_r')

In [ ]:
import tifffile

In [ ]:
class Dataset():
    
    def __init__(
            self, 
            optic_dir,
            sar_dir,
            mask_dir):
        
        self.optic_ids = os.listdir(optic_dir)
        self.sar_ids = os.listdir(sar_dir)
        self.mask_ids = os.listdir(mask_dir)
        
        self.optic_fps = [os.path.join(optic_dir, image_id) for image_id in self.optic_ids]
        self.sar_fps = [os.path.join(sar_dir, image_id) for image_id in self.sar_ids]
        self.masks_fps = [os.path.join(mask_dir, image_id) for image_id in self.mask_ids]
        
    def __getitem__(self, i):
        
        optic_id = self.optic_fps[i].split('\\')[-1]
        sar_id = self.sar_fps[i].split('\\')[-1]
        mask_id = self.masks_fps[i].split('\\')[-1]
        
        optic = tifffile.imread(self.optic_fps[i])
        sar = tifffile.imread(self.sar_fps[i])
        mask = tifffile.imread(self.masks_fps[i])

        return optic, sar, mask
    
    def __len__(self):
        return len(self.mask_ids)

In [ ]:
dataset_dir = r"C:\Users\CBS-7\Desktop\Tez\DATA\patches"

m_dir = os.path.join(dataset_dir, "mask")
o_dir = os.path.join(dataset_dir, "optic")

In [ ]:
viz_dataset = Dataset(optic_dir = o_dir, 
                        sar_dir = s_dir,
                        mask_dir = m_dir)

In [ ]:
optic, sar, mask = viz_dataset[np.random.randint(len(viz_dataset))]


In [ ]:
optic.dtype, optic.shape, optic.max()

In [ ]:
sar.dtype, sar.shape, sar.max()

In [ ]:
mask.dtype, mask.shape, mask.max()

In [ ]:
def scale_range(input, min, max):
    input += -(np.min(input))
    input /= (1e-9 + np.max(input) / (max - min + 1e-9))
    input += min
    return input

optic, sar, mask = viz_dataset[np.random.randint(len(viz_dataset))]

visualize_in_one_row(Image = scale_range(optic.astype('float'), 0, 255)[...,:3].astype(np.uint8) ,
          sar_image = sar,
           mask = mask)

In [ ]:
optic.shape, optic.dtype,sar.shape, sar.dtype, mask.shape, mask.dtype

# SPLIT THE PATCHES INTO TRAIN VAL AND TEST RANDOMLY

## Delete non square and empty masks with sar and optic correspondings 

In [ ]:
import os
import skimage.io
import numpy as np
import glob
import shutil

In [ ]:
#original
def _read_img(image_path):
    img = skimage.io.imread(image_path, plugin='tifffile')
    return img

deleted_ids = []

path = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\mask_patch\tif"
for filename in os.listdir(path):
    img = _read_img(path + '\\' + filename)
    sum_of_pixels = np.sum(img)
    if sum_of_pixels == 0 or img.shape[0] != img.shape[1]:
        last = filename.split('.')[0].split('_')[-1]
        before = filename.split('.')[0].split('_')[-2]
        id_ = before + '_' + last
        deleted_ids.append(id_)
        os.remove(path + '\\' + filename)

In [ ]:
len(deleted_ids) # 18761 -> 7076

In [ ]:
for i in range(len(os.listdir(r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\image_patch\tif"))):
    for fileloc in glob.iglob(r'C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\image_patch\tif\DH_1940_Clipped_{}.tif'.format(deleted_ids[i])):
        os.remove(fileloc)

### Split  

In [ ]:
ids__ = os.listdir(r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\mask_patch\tif")
all_ids = []

for x,i in enumerate(ids__):
    last = ids__[x].split('.')[0].split('_')[-1]
    before = ids__[x].split('.')[0].split('_')[-2]
    id_ = before + '_' + last
    all_ids.append(id_)

In [ ]:
all_ids

In [ ]:
val_ratio = 0.2
test_ratio = 0.1

np.random.seed(5) #5 vardı sanırım 3 için
np.random.shuffle(all_ids)

train_FileNames, val_FileNames, test_FileNames = np.split(np.array(all_ids),
                                                              [int(len(all_ids)* (1 - (val_ratio + test_ratio))), 
                                                               int(len(all_ids)* (1 - test_ratio))])

In [ ]:
len(train_FileNames), len(val_FileNames), len(test_FileNames)

In [ ]:
train_FileNames

In [ ]:
dirr_mask = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\mask_patch\tif"

_all_fn_train = ['GT_colored_{}.tif'.format(name) for name in train_FileNames]
_all_fn_val = ['GT_colored_{}.tif'.format(name) for name in val_FileNames]
_all_fn_test = ['GT_colored_{}.tif'.format(name) for name in test_FileNames]

train_mask = [os.path.join(dirr_mask, _all_fn_train[i]) for i, name in enumerate(train_FileNames)]
val_mask = [os.path.join(dirr_mask, _all_fn_val[i]) for i, name in enumerate(val_FileNames)]
test_mask = [os.path.join(dirr_mask, _all_fn_test[i]) for i, name in enumerate(test_FileNames)]
#train_mask

In [ ]:
dirr_optic = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\image_patch\tif"

all_fn_train_ = ['DH_1940_Clipped_{}.tif'.format(name) for name in train_FileNames]
all_fn_val_ = ['DH_1940_Clipped_{}.tif'.format(name) for name in val_FileNames]
all_fn_test_ = ['DH_1940_Clipped_{}.tif'.format(name) for name in test_FileNames]

train_optic = [os.path.join(dirr_optic, all_fn_train_[i]) for i, name in enumerate(train_FileNames)]
val_optic = [os.path.join(dirr_optic, all_fn_val_[i]) for i, name in enumerate(val_FileNames)]
test_optic = [os.path.join(dirr_optic, all_fn_test_[i]) for i, name in enumerate(test_FileNames)]

In [ ]:
train_optic


In [ ]:
#ORIGINAL ONE

root_dir = r'C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class' # data root path
classes_dir = ['Dataset'] #total labels

val_ratio = 0.2
test_ratio = 0.1

for cls in classes_dir:

    src = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\image_patch\tif"

    allFileNames = os.listdir(src)

    np.random.shuffle(allFileNames)

    train_FileNames = train_optic
    val_FileNames = val_optic
    test_FileNames = test_optic

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, (os.path.join(root_dir,cls,'Train','Image')))

    for name in val_FileNames:
        shutil.copy(name, (os.path.join(root_dir,cls,'Val','Image')))

    for name in test_FileNames:
        shutil.copy(name,(os.path.join(root_dir,cls,'Test','Image')))

In [ ]:
#ORIGINAL ONE

root_dir = r'C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class' # data root path
classes_dir = ['Dataset'] #total labels

val_ratio = 0.2
test_ratio = 0.1

for cls in classes_dir:

    src = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\mask_patch\tif"

    allFileNames = os.listdir(src)

    np.random.shuffle(allFileNames)

    train_FileNames = train_mask
    val_FileNames = val_mask
    test_FileNames = test_mask

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, (os.path.join(root_dir,cls,'Train','Mask')))

    for name in val_FileNames:
        shutil.copy(name, (os.path.join(root_dir,cls,'Val','Mask')))

    for name in test_FileNames:
        shutil.copy(name,(os.path.join(root_dir,cls,'Test','Mask')))

### END


## Dataset Check

In [ ]:
palette ={ 0 : (0,0,0), # NoData
           1 : (77, 255, 0), #Ausgebaute Allwetterstrasse
           2 : (204, 0, 0),  #Eisenbahn
           3 : (230, 128, 0), #Fußweg
           4 : (255, 0, 0),  #Karawanenweg
           5 : (0, 204, 242), #Saumweg
         }

invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(arr_2d, palette=palette):
    """ Numeric labels to RGB-color encoding """
    arr_3d = np.zeros((arr_2d.shape[0], arr_2d.shape[1], 3), dtype=np.uint8)

    for c, i in palette.items():
        m = arr_2d == c
        arr_3d[m] = i

    return arr_3d

def convert_from_color(arr_3d, palette=invert_palette):
    """ RGB-color encoding to grayscale labels """
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    return arr_2d

# helper function for data visualization
def visualize_in_one_row(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
def scale_range(input, min, max):
    input += -(np.min(input))
    input /= (1e-9 + np.max(input) / (max - min + 1e-9))
    input += min
    return input

def show_image(im,target_file): #PCG
      if len(im.shape) == 2:
        im = scale_range(im, 0, 255).astype(np.uint8)
        plt.figure(figsize=(16, 16), dpi= 80, facecolor='w', edgecolor='k')
        plt.imshow(im,cmap='gray')
        plt.show()

      elif len(im.shape) == 3:
        im = np.array([scale_range(i, 0, 255) for i in im.transpose((2,0,1))]).transpose(1,2,0)[...,:3].astype(np.uint8)
        plt.figure(figsize=(16, 16), dpi= 80, facecolor='w', edgecolor='k')
        plt.savefig(target_file, format = 'png', dpi =80,  transparent = True, bbox_inches = 'tight', pad_inches = 0)
        plt.imshow(im)
        plt.show()

In [ ]:
import os
import numpy as np
import tifffile
from matplotlib import pyplot as plt

class Dataset():
    
    def __init__(
            self, 
            dataset_dir):
        
        self.dataset_dir = dataset_dir        
        self.train_dir = os.path.join(self.dataset_dir, 'Train')
            
        optic_dir = os.path.join(self.train_dir, 'Image')
        mask_dir = os.path.join(self.train_dir, 'Mask')
            
        self.optic_ids = os.listdir(optic_dir)
        self.mask_ids = os.listdir(mask_dir)

        self.optic_fps = [os.path.join(optic_dir, image_id) for image_id in self.optic_ids]
        self.mask_fps = [os.path.join(mask_dir, image_id) for image_id in self.mask_ids]
        print("Training set: {} image".format(len(self.optic_ids)))
            
    def __getitem__(self, i):
        
        optic = tifffile.imread(self.optic_fps[i])
        mask = tifffile.imread(self.mask_fps[i])
        
        return optic, mask
    
    def __len__(self):
        
        return len(self.mask_ids)
  

In [ ]:
dataset_dir = r"C:\Users\BURAK\Desktop\YOL\yol_\Multi_Class\Dataset"

viz_dataset = Dataset(dataset_dir = dataset_dir)
mask_container = []

for i in range(len(viz_dataset)):
    optic, mask = viz_dataset[np.random.randint(len(viz_dataset))]
    converted_mask = convert_from_color(mask)
    mask_container.append(converted_mask)
    print(converted_mask)
    #visualize_in_one_row(Optical = scale_range(optic.astype('float'), 0, 255)[...,:3].astype(np.uint8),
    #                      Ground_Truth = mask)

#### COUNT CLASSES 

In [ ]:
masks = np.asarray(mask_container)

In [ ]:
masks.shape

In [ ]:
unique, counts = np.unique(masks, return_counts=True)
unique, counts, len(counts)

In [ ]:
def get_all_ids(path = r"C:\Users\CBS-7\Desktop\Tez\DATA\patches\mask", val_ratio = 0.2, test_ratio = 0.1): 
    ids__ = os.listdir(path)
    all_ids = []

    for x,i in enumerate(ids__):
        last = ids__[x].split('.')[0].split('_')[-1]
        before = ids__[x].split('.')[0].split('_')[-2]
        id_ = before + '_' + last
        all_ids.append(id_)
        
        np.random.shuffle(all_ids)

        train_, val_, test_ = np.split(np.array(all_ids), [int(len(all_ids)* (1 - (val_ratio + test_ratio))), 
                                                                       int(len(all_ids)* (1 - test_ratio))])
        return train_, val_, test_

def get_train_val_test_ids(root, mode = 'mask'):
    
    train__, val__, test__ = get_all_ids()
    print("trrr idss", train__)
    
    if mode == 'mask':
        
        get_im_IDs = os.path.join(root, 'mask')

        all_fn_train = ['MASK_Final_color_CROP_{}.tif'.format(name) for name in train__]
        all_fn_val = ['MASK_Final_color_CROP_{}.tif'.format(name) for name in val__]
        all_fn_test = ['MASK_Final_color_CROP_{}.tif'.format(name) for name in test__]

        train_ims = [os.path.join(get_im_IDs, all_fn_train[i]) for i, name in enumerate(train__)]
        val_ims = [os.path.join(get_im_IDs, all_fn_val[i]) for i, name in enumerate(val__)]
        test_ims = [os.path.join(get_im_IDs, all_fn_test[i]) for i, name in enumerate(test__)]

    if mode == 'optic':

        get_im_IDs = os.path.join(root, 'optic')

        all_fn_train = ['cropped_optic_{}.tif'.format(name) for name in train__]
        all_fn_val = ['cropped_optic_{}.tif'.format(name) for name in val__]
        all_fn_test = ['cropped_optic_{}.tif'.format(name) for name in test__]

        train_ims = [os.path.join(get_im_IDs, all_fn_train[i]) for i, name in enumerate(train__)]
        val_ims = [os.path.join(get_im_IDs, all_fn_val[i]) for i, name in enumerate(val__)]
        test_ims = [os.path.join(get_im_IDs, all_fn_test[i]) for i, name in enumerate(test__)]
        
    if mode == 'SAR':
    
        get_im_IDs = os.path.join(root, 'sar')

        all_fn_train = ['cropped_sar_{}.tif'.format(name) for name in train__]
        all_fn_val = ['cropped_sar_{}.tif'.format(name) for name in val__]
        all_fn_test = ['cropped_sar_{}.tif'.format(name) for name in test__]

        train_ims = [os.path.join(get_im_IDs, all_fn_train[i]) for i, name in enumerate(train__)]
        val_ims = [os.path.join(get_im_IDs, all_fn_val[i]) for i, name in enumerate(val__)]
        test_ims = [os.path.join(get_im_IDs, all_fn_test[i]) for i, name in enumerate(test__)]
        
        return train_ims, val_ims, test_ims

def split_dataset(source_dir, target_dir, modalities):
    

    src = os.path.join(source_dir, modalities) 

    allFileNames = os.listdir(src)

    np.random.shuffle(allFileNames)

    if modalities == 'mask':
        x, y, z = get_train_val_test_ids(root = source_dir, mode='mask')
        
    if modalities == 'optic':
        x, y, z = get_train_val_test_ids(root = source_dir, mode='optic')
            
    if modalities == 'sar':
        x, y, z = get_train_val_test_ids(root = source_dir, mode='sar')

            
    print('Total images: ', len(allFileNames))
    print('Training: ', len(x))
    print('Validation: ', len(y))
    print('Testing: ', len(z))

    # Copy-pasting images
    for name in x:
        shutil.copy(name, (os.path.join(target_dir,'train',modalities)))

    for name in y:
        shutil.copy(name, (os.path.join(target_dir,'val',modalities)))

    for name in z:
        shutil.copy(name,(os.path.join(target_dir,'test',modalities)))
        
        
import os
import skimage.io
import numpy as np
import glob
import shutil

source_dir = r"C:\Users\CBS-7\Desktop\Tez\DATA\patches"
target_dir = r'C:\Users\CBS-7\Desktop\Tez\DATA\dataset2' # Where to save? 

split_dataset(source_dir = source_dir, target_dir = target_dir , modalities = 'mask')